### Нерешенные проблемы

- Я не понимаю как заставить работать функцию "назад"
- Нужно сохранять и подргружать данные из json-а, это легко
- Нужно придумать метрику и логику для статистики
- Нужно оценить размер хранимых данных
- Нужно привязать парсинг и нормальные данные к коду
- Нет приятного оформления интерфейса 
- Аномалии не обработаны 

In [ ]:
import telebot
from telebot import types
import sqlite3
import random

m_m_btn = ['Мой портфель','Биржа','Статистика','Рейтинг'] #Main menu buttons

stocks_btn = ['YNDX','HUH','AAPL','GOOG','AMZN','GACHI','TSLA','IUP','GME'] # Челы закупаемся гачи

clients = dict()

stocks = dict()

stocks['sample_ticker'] = {'current_price' : 5, 'company_name' : '123', 'info' : {}}
stocks['sample_ticker_1'] = {'current_price' : 5, 'company_name' : '123', 'info' : {}}


In [ ]:
def register_client(client_id):
    global clients
    if client_id not in clients:
        clients[client_id] = {'stocks': {},
                              'registration_date': 0,
                              'history': [],
                              'account': 1000000
                             }
        
def register_stock(item):
    global stocks
    if item not in stocks:
        stocks[item] = {'current_price' : random.randint(0,400), 'company_name' : '123', 'info' : {}}
        

def buy(client_id, ticker, quantity):
    global stocks, clients            
    cost = stocks[ticker]['current_price'] * quantity
    if ticker not in clients[client_id]['stocks'].keys():
        clients[client_id]['stocks'][ticker] = {'buy_price': 0, 'quantity': 0}
    clients[client_id]['account'] -= cost
    clients[client_id]['stocks'][ticker]['quantity'] += quantity
            
            
def sell(client_id, ticker, quantity):
    global stocks, clients                                               #возможно while
    cost = stocks[ticker]['current_price'] * quantity 
    if ticker not in clients[client_id]['stocks'].keys():
        clients[client_id]['stocks'][ticker] = {'buy_price': 0, 'quantity': 0}
    clients[client_id]['account'] += cost
    clients[client_id]['stocks'][ticker]['quantity'] -= quantity       


In [ ]:
for item in stocks_btn:
    register_stock(item)
stocks

### Ветка маркета



In [ ]:
def get_operation(msg):
    
    global operation 
    if msg.text == 'Назад':
        bot.register_next_step_handler(msg, callback_worker)
    elif msg.text == 'BUY':

        operation = msg.text
        try:
            keyboard = types.ReplyKeyboardMarkup(row_width=2)
            for i in range(len(stocks_btn)):
                keyboard.add(types.KeyboardButton(stocks_btn[i]))

            msg = bot.send_message(msg.chat.id, text = 'Выберите акцию', reply_markup=keyboard)
            bot.register_next_step_handler(msg, get_ticker)
        
        except:
            pass
        
    elif msg.text == 'SELL':
        operation = msg.text
        try:
            keyboard = types.ReplyKeyboardMarkup(row_width=2)
            for i in range(len(stocks_btn)):
                keyboard.add(types.KeyboardButton(stocks_btn[i]))

            msg = bot.send_message(msg.chat.id, text = 'Выберите акцию', reply_markup=keyboard)
            bot.register_next_step_handler(msg, get_ticker)
        
        except:
            pass
        
        
def get_ticker(msg):
    
    global ticker
    ticker = msg.text
    msg = bot.send_message(msg.chat.id, text = 'Введите количество')
    bot.register_next_step_handler(msg, get_quantity)
        
        
def get_quantity(msg):
    
    global quantity
    quantity = int(msg.text)
    keyboard = types.ReplyKeyboardMarkup(row_width=2)
    keyboard.add(types.KeyboardButton('Назад'),types.KeyboardButton('Подтвердить'))
    msg = bot.send_message(msg.chat.id, text = 'Подтвердите операцию', reply_markup=keyboard)
    bot.register_next_step_handler(msg, get_confirm)
    
    
def get_confirm(msg):

    keyboard = types.ReplyKeyboardMarkup(row_width=1)
    keyboard.add(types.KeyboardButton('В главное меню')) 
    if msg.text == 'Подтвердить':
        
        global quantity, ticker, operation
        
        if operation == 'BUY':
            buy(msg.from_user.id, ticker, quantity)
        elif operation == 'SELL':
            sell(msg.from_user.id, ticker, quantity)
              
        msg = bot.send_message(msg.chat.id, 'Операция успешно совершена!')  
        bot.register_next_step_handler(msg, send_keyboard)
    

In [ ]:
# Функция обработки списка активов челикса в строку

def stocks_to_string(client_id):
    global stocks 
    s = 'Ваши акции: \n'
    for item in sorted(clients[client_id]['stocks'].keys()):
        s += item + ': ' + str(clients[client_id]['stocks'][item]['quantity']) + ' цена: ' + str(stocks[item]['current_price'] )+ '\n'
        
    s =  s + 'Ваш баланс: ' + str(clients[client_id]['account'])
                              
    return s

In [ ]:
clients

In [ ]:
{'stocks': {},
                              'registration_date': 0,
                              'history': [],
                              'account': 1000000
                             }

In [9]:
token ='1577869583:AAGdaTiEB3db1QCj7-HxrGrmK28Ho9rkDa8'

bot = telebot.TeleBot(token)

@bot.message_handler(commands=['start'])
def send_keyboard(msg, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=4)  # наша клавиатура
    
    register_client(msg.from_user.id)
    
    for i in range(len(m_m_btn)):
        keyboard.add(m_m_btn[i])

    # пришлем это все сообщением и запишем выбранный вариант
    msg = bot.send_message(msg.from_user.id,
                     text=text, reply_markup=keyboard)

    # отправим этот вариант в функцию, которая его обработает
    bot.register_next_step_handler(msg, callback_worker)
    
# привязываем функции к кнопкам на клавиатуре

def callback_worker(msg):
    if msg.text == 'Мой портфель':

        s = stocks_to_string(msg.from_user.id)
        keyboard = types.ReplyKeyboardMarkup(row_width=2)
        keyboard.add(types.KeyboardButton('BUY'),types.KeyboardButton('SELL'))
        keyboard.add(types.KeyboardButton('Назад'))  
        msg = bot.send_message(msg.chat.id, text = s, reply_markup=keyboard)
        bot.register_next_step_handler(msg, get_operation)
        
    elif msg.text == 'Биржа':
        
        stocks_string = '\n'.join(stocks_btn)
        s = 'Список акций \n' + stocks_string
        
        keyboard = types.ReplyKeyboardMarkup(row_width=4)
        for i in range(len(stocks_btn) // 2):
            keyboard.add(types.KeyboardButton(stocks_btn[i*2]), types.KeyboardButton(stocks_btn[i*2+1]))
        keyboard.add(types.KeyboardButton('Назад'))        
        msg = bot.send_message(msg.chat.id, text = s, reply_markup=keyboard)
        bot.register_next_step_handler(msg, get_ticker)
            
    elif msg.text == 'Статистика':
        pass

    elif msg.text == 'Рейтинг':
        pass

        
@bot.message_handler(content_types=['text'])
def handle_docs_audio(message):
    send_keyboard(message, text="Я не понимаю :-( Выберите один из пунктов меню:")
    

bot.polling(none_stop=True) # или bot.infinity_polling()

TypeError: send_message() missing 1 required positional argument: 'text'

In [ ]:
type(clients[952705641]['stocks']['YNDX']['quantity'])

In [ ]:
clients[1153306527]['stocks']['GACHI']['quantity'] =0

In [ ]:
def stocks_to_string(client_id):
    s = str()
    for item in sorted(clients[client_id]['stocks'].keys()):
        s += item + ': ' + clients[client_id]['stocks'][item] + '\n'
    return s